In [ ]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.2 MB/s eta 0:00:00


In [ ]:
# ======================================
# AI Resume Screening System
# ======================================

from google.colab import files
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from scipy.sparse import hstack
from sklearn.metrics import accuracy_score
import numpy as np
from PyPDF2 import PdfReader

# ======================================
# SECTION 1: LOAD & PREPARE DATASET
# ======================================

print("📚 Loading dataset...")
df = pd.read_csv('/content/AI_Resume_Screening.csv')

# Handle missing values
df['Skills'] = df['Skills'].fillna('')
df['Certifications'] = df['Certifications'].fillna('NA')

# Combine text features
df['Text_Feature'] = df['Skills'] + " " + df['Certifications']

# Extract numeric features
numeric_features = df[['Experience (Years)', 'Projects Count', 'Salary Expectation ($)']].fillna(0)

print(f"✅ Dataset loaded: {len(df)} records")

# ======================================
# SECTION 2: TEXT VECTORIZATION
# ======================================

print("\n📝 Vectorizing text features...")

# TF-IDF for text features
vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
X_text = vectorizer.fit_transform(df['Text_Feature'])

# Standardize numeric features
scaler = StandardScaler()
X_num = scaler.fit_transform(numeric_features)

# Combine text and numeric features
X = hstack([X_text, X_num])
y = df['Recruiter Decision']

print(f"✅ Features created: {X.shape[1]} total features")

# ======================================
# SECTION 3: TRAIN MODEL
# ======================================

print("\n🤖 Training Random Forest Model...")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model = RandomForestClassifier(
    n_estimators=200,
    max_depth=20,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ Model Accuracy: {accuracy:.2%}")

# ======================================
# SECTION 4: EXTRACT TEXT FROM PDF
# ======================================

def extract_text_from_pdf(file_path):
    """Extract text from PDF using PyPDF2"""
    try:
        text = ""
        with open(file_path, 'rb') as file:
            pdf_reader = PdfReader(file)
            for page in pdf_reader.pages:
                text += page.extract_text() + "\n"
        return text
    except Exception as e:
        print(f"❌ Error reading PDF: {e}")
        return ""

def clean_text(text):
    """Clean and normalize text"""
    text = re.sub(r'•|\-', '\n', text)  # Convert bullets to newlines
    text = re.sub(r'\s+', ' ', text)   # Remove extra spaces
    text = re.sub(r'[^\w\s,./+()&]', ' ', text)  # Remove special chars
    return text.strip()

# ======================================
# SECTION 5: EXTRACT RESUME SECTIONS
# ======================================

def extract_section(text, section_keywords, stop_keywords):
    """Extract specific section from resume"""
    text_lower = text.lower()

    # Find section start
    start_pos = None
    for keyword in section_keywords:
        pos = text_lower.find(keyword)
        if pos != -1:
            start_pos = pos + len(keyword)
            break

    if not start_pos:
        return "Not found"

    # Find section end
    remaining_text = text_lower[start_pos:]
    stop_positions = [remaining_text.find(stop) for stop in stop_keywords
                      if remaining_text.find(stop) != -1]

    end_pos = min(stop_positions) if stop_positions else len(remaining_text)

    # Extract and clean
    section_text = remaining_text[:end_pos]
    section_text = re.sub(r'[^a-z0-9,./+()& ]', ' ', section_text)
    section_text = re.sub(r'\s+', ' ', section_text)

    return section_text.strip() if section_text else "Not found"

# ======================================
# SECTION 6: UPLOAD & PROCESS RESUME
# ======================================

print("\n📂 Upload your resume (PDF format only)...")
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# Extract text from PDF
raw_text = extract_text_from_pdf(file_name)
text_clean = clean_text(raw_text)

print("✅ Resume text extracted successfully!\n")

# Define section keywords
SKILLS_KEYWORDS = ["skills", "technical skills", "key skills", "core skills"]
CERT_KEYWORDS = ["certifications", "certification", "courses", "training", "achievements"]
STOP_KEYWORDS = ["languages", "interests", "education", "projects", "experience", "summary"]

# Extract sections
skills_text = extract_section(text_clean, SKILLS_KEYWORDS, STOP_KEYWORDS)
cert_text = extract_section(text_clean, CERT_KEYWORDS, STOP_KEYWORDS)

# Parse into lists
skills_list = [s.strip() for s in re.split(r'[,:;/\n]', skills_text)
               if len(s.strip()) > 2]
cert_list = [c.strip() for c in re.split(r'[,:;/\n]', cert_text)
             if len(c.strip()) > 2]

print(f"🧠 Extracted {len(skills_list)} Skills: {skills_list[:5]}...")
print(f"📜 Extracted {len(cert_list)} Certifications: {cert_list[:5]}...")

# ======================================
# SECTION 7: COLLECT USER INPUT
# ======================================

print("\n📋 Enter your resume details:\n")

# Job Role Selection
job_roles = {
    '1': 'AI Researcher',
    '2': 'Data Scientist',
    '3': 'Cybersecurity Analyst',
    '4': 'Software Engineer'
}

print("🎯 Select Job Role:")
for key, role in job_roles.items():
    print(f"   {key}. {role}")

while True:
    role_choice = input("\nEnter role number (1-4): ").strip()
    if role_choice in job_roles:
        selected_role = job_roles[role_choice]
        print(f"✅ Selected: {selected_role}\n")
        break
    else:
        print("❌ Invalid choice. Please enter 1-4.\n")

try:
    experience = float(input("👤 Years of Experience: "))
    projects_count = int(input("💼 Number of Projects: "))
    salary_expect = float(input("💰 Salary Expectation ($): "))
except ValueError:
    print("❌ Invalid input. Using default values.")
    experience, projects_count, salary_expect = 0, 0, 0

# ======================================
# SECTION 8: PREPARE FEATURES & PREDICT
# ======================================

print("\n🔄 Processing your application...")

# Create feature vector
resume_combined_text = " ".join(skills_list + cert_list)
X_text_new = vectorizer.transform([resume_combined_text])
X_num_new = scaler.transform([[experience, projects_count, salary_expect]])
X_new = hstack([X_text_new, X_num_new])

# Make prediction
decision = model.predict(X_new)[0]
decision_proba = model.predict_proba(X_new)[0]

# ======================================
# SECTION 9: DISPLAY RESULTS
# ======================================

print("\n" + "="*50)
print("📊 PREDICTION RESULTS")
print("="*50)
print(f"🎯 Applied Role: {selected_role}")
print(f"💼 Decision: {'✅ SELECTED' if decision == 1 else '❌ REJECTED'}")
print(f"📈 Confidence: {max(decision_proba):.1%}")
print("="*50 + "\n")

# Optional: Show feature importance
if len(skills_list) > 0:
    print(f"✨ Top Skills Detected: {', '.join(skills_list[:3])}")
if len(cert_list) > 0:
    print(f"🏆 Certifications: {', '.join(cert_list[:3])}")
print(f"📅 Experience: {experience} years")
print(f"💰 Salary Expectation: ${salary_expect:,.0f}")

📚 Loading dataset...
✅ Dataset loaded: 1000 records

📝 Vectorizing text features...
✅ Features created: 24 total features

🤖 Training Random Forest Model...
✅ Model Accuracy: 94.50%

📂 Upload your resume (PDF format only)...


Saving satyam final resume.pdf to satyam final resume.pdf
✅ Resume text extracted successfully!

🧠 Extracted 6 Skills: ['frontend development html5', 'css3', 'javascript', 'react', 'tailwind css']...
📜 Extracted 2 Certifications: ['dataplus overview certification tcs mastercraft', 'april 2023 april 2025 power bi course satish dhwale']...

📋 Enter your resume details:

🎯 Select Job Role:
   1. AI Researcher
   2. Data Scientist
   3. Cybersecurity Analyst
   4. Software Engineer

Enter role number (1-4): 2
✅ Selected: Data Scientist

👤 Years of Experience: 1
💼 Number of Projects: 1
💰 Salary Expectation ($): 100

🔄 Processing your application...

📊 PREDICTION RESULTS
🎯 Applied Role: Data Scientist
💼 Decision: ❌ REJECTED
📈 Confidence: 73.0%

✨ Top Skills Detected: frontend development html5, css3, javascript
🏆 Certifications: dataplus overview certification tcs mastercraft, april 2023 april 2025 power bi course satish dhwale
📅 Experience: 1.0 years
💰 Salary Expectation: $100


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
